# Import Libraries





In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [2]:
#import libraries
import numpy as np
import pandas as pd
import pickle
#from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import cross_validate,cross_val_score,cross_val_predict,train_test_split,StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
from source.utils.config import *
from source.utils.metrics_utils import *

# Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variable in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.

In [3]:
dataset_name = 'diabetes-women' #options: "diabetes-women", "sepsis", 'aids', "myocardial-infarction", 'alzheimer-disease', "diabetes-prediction", "stroke-prediction"
mitigation = 'original'

In [4]:
dataset_config = datasets_config[dataset_name]
ignore_cols = dataset_config['ignore_cols']
target_variable = dataset_config['target_variable']
target_variable_labels = dataset_config['target_variable_labels']
sensible_attributes = dataset_config['sensible_attributes']
default_mappings = dataset_config.get('default_mappings', {})
n_splits = dataset_config['n_splits']

In [5]:
dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

# Compute measures


In [6]:
for sensible_attribute in sensible_attributes:
  print(dataset_name, sensible_attribute)
  predictions_and_tests = compute_predictions_and_tests(df, sensible_attribute, target_variable, n_splits, models, n_estimators, random_seed)
  performance_metrics = compute_performance_metrics(predictions_and_tests, models, n_splits, mitigation)

  print(performance_metrics)
  save_path = path_to_project + '/measurements/performance_metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(performance_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

diabetes-women AgeCategory


  0%|          | 0/6 [00:00<?, ?it/s]

{'accuracy': {'Logistic Regression': [0.7792592592592593, 0.040409427250328485], 'Decision Tree': [0.6992592592592592, 0.030934241507883122], 'Bagging': [0.7614814814814814, 0.047638326491299336], 'Random Forest': [0.762962962962963, 0.025228720542113222], 'Extremely Randomized Trees': [0.7614814814814814, 0.028264865227168756], 'Ada Boost': [0.7555555555555555, 0.03072065385678182]}, 'precision': {'Logistic Regression': [0.7160834688602383, 0.04346956697678775], 'Decision Tree': [0.5491448445171849, 0.07724421383826428], 'Bagging': [0.7266390121228831, 0.03597203484255246], 'Random Forest': [0.6882497826716893, 0.075840460958793], 'Extremely Randomized Trees': [0.6745220057720058, 0.04616237016205394], 'Ada Boost': [0.6468776882900535, 0.03685532599107633]}, 'recall': {'Logistic Regression': [0.5528883981398508, 0.04445903125740144], 'Decision Tree': [0.562950686082323, 0.05687042139381678], 'Bagging': [0.45765937906791443, 0.07347018281503975], 'Random Forest': [0.5216634802228872, 0

In [9]:
for sensible_attribute in sensible_attributes:
  print(dataset_name, sensible_attribute)
  predictions_and_tests = compute_predictions_and_tests(df, sensible_attribute, target_variable, n_splits, models, n_estimators, random_seed)
  print(predictions_and_tests)
  fairness_metrics = compute_fairness_metrics(predictions_and_tests, target_variable_labels, models, n_splits, mitigation)
  print(fairness_metrics)
  final_metrics = compute_mean_std_dev_fairness_metrics(fairness_metrics, models, n_splits, mitigation)
  print(final_metrics)

  save_path = path_to_project + '/measurements/metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
      pickle.dump(final_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

diabetes-women AgeCategory


  0%|          | 0/6 [00:00<?, ?it/s]

{'Logistic Regression': {0: {'y_test': array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1]), 'y_pred': array([1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1]), 's_test': array([1, 1, 1, 0, 1, 1, 1, 1

# Check measures

In [ ]:
print("PREDS")
for i in range(0,n_splits):
  print(i)
  for m in models:
    print(m, predictions_and_tests[m][i]['y_pred'])
  print('---------------')


PREDS
0
Logistic Regression [1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0
 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
Decision Tree [0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0]
Bagging [1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0
 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Random Forest [1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0
 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0]
Extremely Randomized Trees [0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0
 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
Ada Boost [0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 0
 1 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0]
---------------
1
Logistic Regression [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 

In [ ]:
print("DIVISION")
for metric in fairness_catalogue:
  print(metric)
  for m in models:
    print(m)
    for i in range(0,n_splits):
      print(i, fairness_metrics["division"][m][i][metric]['Value'])
  print('---------------')

DIVISION
GroupFairness
Logistic Regression
0 0.5576923076923079
1 0.4516129032258065
2 0.5157894736842104
3 0.3105413105413104
4 0.6666666666666667
5 0.4245014245014245
6 0.3999999999999999
7 0.3138888888888889
8 0.431111111111111
9 0.5340909090909092
Decision Tree
0 0.5428571428571429
1 0.4516129032258063
2 0.36032388663967607
3 0.5555555555555556
4 0.41608391608391626
5 0.1358024691358024
6 0.3066666666666664
7 0.34375
8 0.512
9 0.36363636363636376
Bagging
0 0.27810650887573973
1 0.16129032258064524
2 0.29924812030075176
3 0.15740740740740744
4 0.4318181818181819
5 0.23456790123456783
6 0.35483870967741926
7 0.15931372549019618
8 0.28800000000000003
9 0.16883116883116878
Random Forest
0 0.6079404466501241
1 0.3548387096774195
2 0.5680580762250453
3 0.4197530864197532
4 0.5
5 0.4266117969821672
6 0.19225806451612892
7 0.4077380952380951
8 0.26
9 0.17768595041322333
Extremely Randomized Trees
0 0.4660633484162897
1 0.5307917888563047
2 0.3853989813242784
3 0.6835016835016834
4 0.5
5 0.

In [ ]:
print("DIVISION")
for metric in fairness_catalogue:
  print(metric)
  for m in models:
    print(m, final_metrics["division"][m][metric][0])
  print('---------------')

print("SUBTRACTION")
for metric in fairness_catalogue:
  print(metric)
  for m in models:
    print(m, final_metrics["subtraction"][m][metric][0])
  print('---------------')

DIVISION
GroupFairness
Logistic Regression 0.4605894995402635
Decision Tree 0.398828890380093
Bagging 0.2533422046216078
Random Forest 0.39148842261219563
Extremely Randomized Trees 0.4469316768191183
Ada Boost 0.515550972153679
---------------
PredictiveParity
Logistic Regression 0.251475751585726
Decision Tree 0.3491342629683879
Bagging 0.3022559858143382
Random Forest 0.26462512034039165
Extremely Randomized Trees 0.2566673807332446
Ada Boost 0.30489775384424733
---------------
PredictiveEquality
Logistic Regression 0.4883333333333333
Decision Tree 0.39084656084656083
Bagging 0.2688257575757576
Random Forest 0.3971921596921598
Extremely Randomized Trees 0.5306060606060605
Ada Boost 0.3601984126984127
---------------
EqualOpportunity
Logistic Regression -0.8981180223285486
Decision Tree -0.4582536698890082
Bagging -0.2750877192982456
Random Forest -0.7037297789929369
Extremely Randomized Trees -0.7955122655122654
Ada Boost -0.6606092883724461
---------------
EqualizedOdds
Logistic Re

In [ ]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67): #update 67!
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])